# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv", index_col = "City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,port elizabeth,-33.9180,25.5701,21.17,69,26,3.58,ZA,1702220932
1,hawaiian paradise park,19.5933,-154.9731,22.80,98,100,2.06,US,1702220918
2,cafarnaum,-11.6936,-41.4683,33.11,28,0,4.12,BR,1702221259
3,queenstown,-31.8976,26.8753,11.80,86,100,3.91,ZA,1702221259
4,port-aux-francais,-49.3500,70.2167,3.20,74,33,11.88,TF,1702220896


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600,
    color = "City",
    alpha = 0.5
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
# I know the challenge page had different filters than below, but that only gave 7 rows.
# I changed them to have more rows in the dataset.
city_data_df_narrowed = city_data_df.loc[
    (city_data_df["Max Temp"] < 30) &
    (city_data_df["Max Temp"] > 20) &
    (city_data_df["Wind Speed"] < 6) &
    (city_data_df["Cloudiness"] == 0),
    :
]

# Drop any rows with null values
city_data_df_narrowed = city_data_df_narrowed.dropna()

# Display sample data
city_data_df_narrowed

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
28,coronel suarez,-37.4547,-61.9334,28.96,18,0,1.84,AR,1702221268
37,kapa'a,22.0752,-159.3190,24.18,84,0,5.14,US,1702220890
52,wailua homesteads,22.0669,-159.3780,23.44,84,0,5.14,US,1702220980
75,tazacorte,28.6290,-17.9293,24.53,80,0,2.06,ES,1702221287
96,sur,22.5667,59.5289,24.35,49,0,3.03,OM,1702221294
115,lihue,21.9789,-159.3672,23.77,76,0,5.14,US,1702220905
136,veinticinco de mayo,-35.4323,-60.1727,28.32,50,0,1.34,AR,1702221308
153,san jose del cabo,23.0589,-109.6977,21.52,65,0,5.14,MX,1702221315
183,brisas de zicatela,15.8369,-97.0419,27.29,69,0,3.09,MX,1702221327


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_narrowed.loc[
    :,
    ["City", "Country", "Lat", "Lng", "Humidity"]
]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
28,coronel suarez,AR,-37.4547,-61.9334,18,
37,kapa'a,US,22.0752,-159.3190,84,
52,wailua homesteads,US,22.0669,-159.3780,84,
75,tazacorte,ES,28.6290,-17.9293,80,
96,sur,OM,22.5667,59.5289,49,
115,lihue,US,21.9789,-159.3672,76,
136,veinticinco de mayo,AR,-35.4323,-60.1727,50,
153,san jose del cabo,MX,23.0589,-109.6977,65,
183,brisas de zicatela,MX,15.8369,-97.0419,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20,
    "lang": "en"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
coronel suarez - nearest hotel: La Casa Hotel Boutique
kapa'a - nearest hotel: Pono Kai Resort
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
tazacorte - nearest hotel: App Leyma
sur - nearest hotel: Sur Hotel
lihue - nearest hotel: Kauai Palms
veinticinco de mayo - nearest hotel: Jorly
san jose del cabo - nearest hotel: Hotel Posada Terranova
brisas de zicatela - nearest hotel: Casa de Olas
viedma - nearest hotel: Níjar
mukalla - nearest hotel: فندق الجابري
thatta - nearest hotel: Jahanzeb
hafar al-batin - nearest hotel: فندق الفرحان
gunnedah - nearest hotel: The Court House
salalah - nearest hotel: Muscat International Hotel
port hedland - nearest hotel: The Esplanade Hotel
carnarvon - nearest hotel: No hotel found
howrah - nearest hotel: Sun India Guest House
ataq - nearest hotel: الشارقة بلازا
karratha - nearest hotel: Karratha International Hotel
tsiombe - nearest hotel: No hotel found
kailua-kona - nearest hotel: Kona Seaside Hotel
mor

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
28,coronel suarez,AR,-37.4547,-61.9334,18,La Casa Hotel Boutique
37,kapa'a,US,22.0752,-159.3190,84,Pono Kai Resort
52,wailua homesteads,US,22.0669,-159.3780,84,Hilton Garden Inn Kauai Wailua Bay
75,tazacorte,ES,28.6290,-17.9293,80,App Leyma
96,sur,OM,22.5667,59.5289,49,Sur Hotel
115,lihue,US,21.9789,-159.3672,76,Kauai Palms
136,veinticinco de mayo,AR,-35.4323,-60.1727,50,Jorly
153,san jose del cabo,MX,23.0589,-109.6977,65,Hotel Posada Terranova
183,brisas de zicatela,MX,15.8369,-97.0419,69,Casa de Olas


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    alpha = 0.5,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)